In [1]:
import os
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()

#from sklearn.metrics import roc_curve, roc_auc_score, classification_report, accuracy_score
from sklearn.metrics import accuracy_score, roc_curve
import numpy as np

import sys
import time
repo_root = "/data/quo.vadis/"
sys.path.append(repo_root)
from models import CompositeClassifier

/home/dmitrijs.trizna/.pyenv/versions/3.8-dev/lib/python3.8/site-packages/xgboost/compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


## Composite preprocessing of adversarial set

In [2]:
adversarial_testset_path = repo_root + "data/adversarial.emulation.dataset/reports_ember"
adversarial_testset_files = os.listdir(adversarial_testset_path)
adversarial_reports = [x.rstrip(".json") for x in adversarial_testset_files if x.endswith(".json")]
print("Successfull adversarial emulation reports: ", len(adversarial_reports))
adversarial_errors = [x for x in adversarial_testset_files if x.endswith(".err")]
print("Errored adversarial emulation reports: ", len(adversarial_errors))
print("Total adversarial samples: ", len(adversarial_errors) + len(adversarial_reports))
print(f"Emulation success rate: {len(adversarial_reports)/(len(adversarial_reports)+len(adversarial_errors))*100:.2f}%")

Successfull adversarial emulation reports:  5399
Errored adversarial emulation reports:  3497
Total adversarial samples:  8896
Emulation success rate: 60.69%


### NOTE - there is less total samples because we skipped benignware and malware that was already evasive!

Total file number match with filesystem data:
```
/data/quo.vadis/adversarial/samples_adversarial_testset_gamma_ember]$ find . -type f | wc -l
8896
```

Adversarial length match with report_db in emulation module - it loads reports from `data/adversarial.emulation.dataset/reports_ember` since passed as parameter:

In [3]:
a = CompositeClassifier(modules=["emulation"], repo_root=repo_root,
                        emulation_report_path="data/adversarial.emulation.dataset/reports_ember",)
len(a.modules["emulation"].report_db)

5399

In [8]:
# python3 early_fusion_pass.py
x_adv_ember = np.load(repo_root+"evaluation/adversarial/composite_adversarial_evaluation/ember_15sections_10population/X-gamma-vs-ember-early-fusion-pass.arr") 
x_orig_ember = np.load(repo_root+"evaluation/adversarial/composite_adversarial_evaluation/ember_15sections_10population/X-gamma-vs-ember-early-fusion-pass-orig.arr")

# python3 run_ember_pass.py
y_ember_orig = np.load(repo_root+"evaluation/adversarial/composite_adversarial_evaluation/ember_15sections_10population/y-gamma-vs-ember-scores-orig.arr")
y_ember_adv = np.load(repo_root+"evaluation/adversarial/composite_adversarial_evaluation/ember_15sections_10population/y-gamma-vs-ember-scores.arr")
EMBER_THRESHOLD = 0.8336
y_ember_orig_int = (y_ember_orig > EMBER_THRESHOLD).astype(int)
y_ember_adv_int = (y_ember_adv > EMBER_THRESHOLD).astype(int)

x_train = np.load(repo_root+"evaluation/composite/X-1647041985-early-fusion-vectors-train.arr")
y_train = np.load(repo_root+"evaluation/composite/y-1647041985-train.arr")

x_test = np.load(repo_root+"evaluation/composite/X-1647097165-early-fusion-vectors-test.arr")
y_test = np.load(repo_root+"evaluation/composite/y-1647097165-test.arr")

## Different modules

In [9]:
def fit(models, x_trains, y_train, save=False):
    for model in models:
        print(f"training late fusion model for {model}...")
        now = time.time()
        models[model].fit(x_trains[model], y_train)
        print(f"training done for {model}... took: {time.time()-now:.2f}s")
        if save:
            os.makedirs("late_fusion_model_fit", exist_ok=True)
            models[model].save_late_fusion_model(filename="late_fusion_model_fit/"+model)
    return models

In [13]:
# need to modify report path to original dataset
modulelist = [["malconv"], ["filepaths"], ["emulation"], ["ember"],
            ["ember", "emulation"],
            ["ember", "filepaths", "emulation"],
            ["malconv", "ember", "filepaths", "emulation"]]
models = {}
x_trains = {}
x_tests = {}

x_ember_orig = {}
x_ember_adv = {}

for modules in modulelist:
    name = "_".join(modules)
    if len(modules) == 4:
        name = "all"

    models[name] = CompositeClassifier(modules=modules, late_fusion_model="LogisticRegression", root=repo_root)
    x_trains[name] = models[name].get_modular_x(modules, x_train)
    x_tests[name] = models[name].get_modular_x(modules, x_test)
    
    x_ember_orig[name] = models[name].get_modular_x(modules, x_orig_ember)
    x_ember_adv[name] = models[name].get_modular_x(modules, x_adv_ember)
    
    if "ember" in modules:
        ember_index = modules.index("ember")
        # replace ember column with y pass
        x_ember_orig[name][:,ember_index] = y_ember_orig
        x_ember_adv[name][:,ember_index] = y_ember_adv

# Remember: this .fit() really trains only late fusion model
models = fit(models, x_trains, y_train, save=False)

training late fusion model for malconv...
training done for malconv... took: 0.19s
training late fusion model for filepaths...
training done for filepaths... took: 0.23s
training late fusion model for emulation...
training done for emulation... took: 0.30s
training late fusion model for ember...
training done for ember... took: 0.26s
training late fusion model for ember_emulation...
training done for ember_emulation... took: 0.29s
training late fusion model for ember_filepaths_emulation...
training done for ember_filepaths_emulation... took: 0.34s
training late fusion model for all...
training done for all... took: 0.38s


## Comparing

In [18]:
def get_metrics_adv_nonadv(model, x_test, y_test, x_adv, y_adv):
    probs = model.predict_proba(x_test)[:,1]
    probs_adv = model.predict_proba(x_adv)[:,1]
    
    preds = np.where(probs > 0.5, 1, 0)
    preds_adv = np.where(probs_adv > 0.5, 1, 0)

    print("Non-Adversarial Set accuracy:", end=" ")
    print(accuracy_score(y_test, preds))
    #print(classification_report(y_test, preds, zero_division=0))
    
    print("Adversarial Set accuracy:", end="     ")
    print(accuracy_score(y_adv, preds_adv))
    #print(classification_report(y_adv, preds_adv, zero_division=0))
    return probs, probs_adv

y_adv = np.ones(len(x_adv_ember))

print("====== ember (secml) ======")
print("Non-Adversarial Set accuracy:", end=" ")
print(accuracy_score(y_adv, y_ember_orig_int))
print("Adversarial Set accuracy:", end="     ")
print(accuracy_score(y_adv, y_ember_adv_int))

probbs, probbs_adv = {}, {}
for model in models:
    x_orig_t = x_ember_orig[model]
    x_adv_t = x_ember_adv[model]
    print("\n", "="*6, model, "="*6)
    probbs[model], probbs_adv[model] = get_metrics_adv_nonadv(models[model], x_orig_t, y_adv, x_adv_t, y_adv)
    # get_metrics_adv_nonadv(models[model], x_test_t, y_test, x_adv_t, y_adv) if you want against full set

====== ember (secml) ======
Non-Adversarial Set accuracy: 0.9814780514910169
Adversarial Set accuracy:     0.7193924800889053

 ====== malconv ======
Non-Adversarial Set accuracy: 0.9855528801629931
Adversarial Set accuracy:     0.9803667345804778

 ====== filepaths ======
Non-Adversarial Set accuracy: 0.9781441007593998
Adversarial Set accuracy:     0.9781441007593998

 ====== emulation ======
Non-Adversarial Set accuracy: 0.9955547323578441
Adversarial Set accuracy:     0.9764771253935914

 ====== ember ======
Non-Adversarial Set accuracy: 1.0
Adversarial Set accuracy:     0.8705315799222079

 ====== ember_emulation ======
Non-Adversarial Set accuracy: 0.9990739025745509
Adversarial Set accuracy:     0.9562882015187998

 ====== ember_filepaths_emulation ======
Non-Adversarial Set accuracy: 0.9887016114095203
Adversarial Set accuracy:     0.9851824411928135

 ====== all ======
Non-Adversarial Set accuracy: 0.9887016114095203
Adversarial Set accuracy:     0.9851824411928135


In [19]:
print(f" {((y_ember_orig_int).astype(int) == 0).sum()}: ember (secml) classifies as benign in orig malware set")
print(f" {((y_ember_adv_int).astype(int) == 0).sum()}: ember (secml) classifies as benign in adversarial malware set")
evasive = ((y_ember_adv_int).astype(int) == 0).sum() - ((y_ember_orig_int).astype(int) == 0).sum()
evasive_ratio = evasive*100/len(y_ember_adv_int)
print(f" {evasive}, {evasive_ratio:.2f}%: evasive samples and ratio against ember")
print()

for model in models:
    orig_benign = ((probbs[model] > 0.5).astype(int) == 0).sum()
    adv_benign = ((probbs_adv[model] > 0.5).astype(int) == 0).sum()
    evasive = adv_benign - orig_benign
    evasive_ratio = evasive*100/len(probbs[model])
    print(f" {orig_benign}: {model} classifies as benign in orig malware set")
    print(f" {adv_benign}: {model} classifies as benign in adversarial malware set")
    print(f" {evasive}, {evasive_ratio:.2f}%: evasive samples and ratio against {model} ")
    print()

 100: ember (secml) classifies as benign in orig malware set
 1515: ember (secml) classifies as benign in adversarial malware set
 1415, 26.21%: evasive samples and ratio against ember

 78: malconv classifies as benign in orig malware set
 106: malconv classifies as benign in adversarial malware set
 28, 0.52%: evasive samples and ratio against malconv 

 118: filepaths classifies as benign in orig malware set
 118: filepaths classifies as benign in adversarial malware set
 0, 0.00%: evasive samples and ratio against filepaths 

 24: emulation classifies as benign in orig malware set
 127: emulation classifies as benign in adversarial malware set
 103, 1.91%: evasive samples and ratio against emulation 

 0: ember classifies as benign in orig malware set
 699: ember classifies as benign in adversarial malware set
 699, 12.95%: evasive samples and ratio against ember 

 5: ember_emulation classifies as benign in orig malware set
 236: ember_emulation classifies as benign in adversarial

## ROC curves (DRAFT, might no be needed)

Possible only if - ROC evaluation if forming a validation set with benign labels, but replacing original malware with adversarial samples if they were acquired.

In [88]:
modulelist = [["malconv"], ["ember"], ["emulation"], ["malconv", "ember", "filepaths", "emulation"]]
mmodels = {}
for x in modulelist:
    key = "_".join(x)
    if key == "malconv_ember_filepaths_emulation":
        key = "All"
    mmodels[key] = models[key]

def evaluate_adversarial_robustness(models, x_tests, x_tests_adv, y_test, y_test_adv, ax=None):
    probs = {}
    probs_adv = {}

    model = "No Skill"
    most_common_label = np.argmax(np.bincount(y_test.astype(int)))
    probs[model] = np.array([most_common_label for _ in range(len(y_test))], dtype=int)
    probs_adv[model] = np.array([most_common_label for _ in range(len(y_test_adv))], dtype=int)
    
    _, ax = plt.subplots(2, 2, figsize=(14,12))
    ax_idx = {0: [0,0], 1: [0,1], 2: [1,0], 3:[1,1], 4:[2,0], 5:[2,1]}
    
    for i, model in enumerate(models):
        i1, i2 = ax_idx[i][0],ax_idx[i][1]
        probs[model] = models[model].predict_proba(x_tests[model])[:,1]
        probs_adv[model] = models[model].predict_proba(x_tests_adv[model])[:,1]
        # preds = np.where(probs[model] > 0.5, 1, 0)
        # preds_adv = np.where(probs_adv[model] > 0.5, 1, 0)
        
        fpr, tpr, _ = roc_curve(y_test, probs[model])
        fpr_adv, tpr_adv, _ = roc_curve(y_test_adv, probs_adv[model])
        # plot the roc curve for the model
        linestyle = "--" if model == "No Skill" else "solid"
        ax[i1,i2].plot(fpr, tpr, linestyle=linestyle, label=model)
        ax[i1,i2].plot(fpr_adv, tpr_adv, linestyle=linestyle, label=model)
        # axis labels
        ax[i1,i2].set_xlabel('False Positive Rate')
        ax[i1,i2].set_ylabel('True Positive Rate')
        ax[i1,i2].title.set_text(model)
        _ = ax[i1,i2].legend(["regular", "adversarial"])
        # TBD - set legend position

# Evaluation of different attack -- per number of sections